In [ ]:
!pip install geemap &> install.log

In [ ]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 59.3 MB/s eta 0:00:00


In [1]:
import ee
import json
import datetime
import pandas as pd
#import geemap
import os
import math
import geopandas as gpd
import re
import numpy as np

In [2]:
try:
    ee.Initialize()
except:
    ee.Authenticate(auth_mode='notebook')
    ee.Initialize(project='potential-evap')

In [3]:
def longest_numeric_substring(string):
    return max(re.findall(r'\d+', string), key=len)

In [24]:
ET_ts_path = 'ET_TS/'
ET_ts_exported = []
for file in os.listdir(ET_ts_path):
  file_path = os.path.join(ET_ts_path,file)
  ET_ts_exported.append(file_path[-11:-4])

print(len(ET_ts_exported))

9109


In [23]:
shapefiles_list = []
path = "../Shapefiles/GAGES_smooth_shp/"
count_tot_shps = 0
count_shps_not_exp = 0

for file in os.listdir(path):
  if file.endswith('.shp'):
    count_tot_shps += 1
    file_path = os.path.join(path,file)
    basin_no = file_path[-11:-4] #string
    if (basin_no not in ET_ts_exported):
      count_shps_not_exp += 1
      shapefiles_list.append(file_path)

print(len(shapefiles_list))

0


In [20]:
bounds = None
def region_et(image):
    # reduction function
    et = image.reduceRegion( #zonal statistics
        reducer = ee.Reducer.mean(),
        geometry = bounds,
        scale = 1000 # 1 km
    )
    # set the result as a metadata property in the image
    return image.set(et)

In [21]:
for i in range(len(shapefiles_list)):
# for i in range(1):
  shapefile = gpd.read_file(shapefiles_list[i])

  features = []
  for j in range(shapefile.shape[0]):
    geom = shapefile.iloc[j:j+1,:]
    jsonDict = eval(geom.to_json())
    geojsonDict = jsonDict['features'][0]
    features.append(ee.Feature(geojsonDict))

  fc = ee.FeatureCollection(features)
  bounds = fc.geometry().bounds()

  mod16 = ee.ImageCollection("MODIS/061/MOD16A2GF").select("ET").filterBounds(bounds)
  # apply the function and filter for images that were not all masked
  ET_region = mod16.map(region_et).filter(ee.Filter.neq("ET",None))

  timeseries = ET_region.aggregate_array("ET").getInfo()
  timestamp = ET_region.aggregate_array("system:time_start").getInfo()
  dates = pd.to_datetime(np.array(timestamp)*1e6)
  region_series = pd.Series(timeseries,index=dates,name="Region ET [kg/m^2/8day]")

  #-----------------------
  region_series_df = pd.DataFrame(region_series)

  # df.plot(figsize=(15,11))
  file_PathName = 'ET_TS/' + 'basin_' + longest_numeric_substring(shapefiles_list[i]) + '.csv'
  print(i, '.) ', shapefiles_list[i][-11:-4])
  region_series_df.to_csv(path_or_buf=file_PathName,index=True)

0 .)  9312800
1 .)  1475548
2 .)  3282290
3 .)  9486590
4 .)  1088500
5 .)  1365500
6 .)  1364959
7 .)  6880500
8 .)  0028500
9 .)  1310000
10 .)  2367900
11 .)  0061800
12 .)  4059500
13 .)  6884000
14 .)  9415515
15 .)  8052780
16 .)  4107000
17 .)  2294650
18 .)  1516530
19 .)  1541000
20 .)  0128500
21 .)  5261000
22 .)  1545500
23 .)  1162720
24 .)  6799460
25 .)  3289500
26 .)  9489700
27 .)  5451900
28 .)  9311500
29 .)  8177500
30 .)  7111000
31 .)  3209300
32 .)  9409100
33 .)  4087240
34 .)  6422500
35 .)  8173000
36 .)  9444000
37 .)  6795500
38 .)  2413130
39 .)  3060000
40 .)  7020550
41 .)  5517000
42 .)  3400680
43 .)  1585095
44 .)  1237000
45 .)  5588720
46 .)  1369500
47 .)  1438500
48 .)  2341910
49 .)  7257006
50 .)  5417000
51 .)  0155500
52 .)  1462500
53 .)  8178800
54 .)  5287890
55 .)  5413500
56 .)  4124500
57 .)  2221000
58 .)  2092500
59 .)  2201960
60 .)  3047000
61 .)  6183850
62 .)  2181100
63 .)  2148300
64 .)  2225500
65 .)  7312200
66 .)  6501000
67 .)

/var/folders/59/d0zvy9t11r7gcr87xn39xwy00000gn/T/ipykernel_2397/3335560068.py:22: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  region_series = pd.Series(timeseries,index=dates,name="Region ET [kg/m^2/8day]")


120 .)  2336030
121 .)  7172000
122 .)  9276000
123 .)  8010000
124 .)  3194700
125 .)  7062575
126 .)  2294491
127 .)  7072000
128 .)  8014500
129 .)  8059400
130 .)  3159000
131 .)  4289000
132 .)  4077400
133 .)  5302000
134 .)  1377500
135 .)  5435935
136 .)  1426500
137 .)  6295250
138 .)  6784800
139 .)  4240200
140 .)  2110701
141 .)  2084160
142 .)  2455900
143 .)  1422000
144 .)  5202000
145 .)  1153700
146 .)  1435100
147 .)  5311400
148 .)  1089100
149 .)  2105000
150 .)  7316500
151 .)  9112500
152 .)  6174000
153 .)  6445980
154 .)  2101500
155 .)  6625000
156 .)  6501200
157 .)  7325860
158 .)  1109070
159 .)  7216500
160 .)  7188838
161 .)  3339000
162 .)  3055250
163 .)  1646500
164 .)  3018350
165 .)  7247015
166 .)  7195855
167 .)  6930000
168 .)  1097300
169 .)  1466200
170 .)  9486800
171 .)  9489500
172 .)  8177700
173 .)  2115860
174 .)  5515080
175 .)  2291500
176 .)  3244000
177 .)  9444200
178 .)  1042800
179 .)  1673550
180 .)  3415000
181 .)  3284230
182 .)  

/var/folders/59/d0zvy9t11r7gcr87xn39xwy00000gn/T/ipykernel_2397/3335560068.py:22: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  region_series = pd.Series(timeseries,index=dates,name="Region ET [kg/m^2/8day]")


430 .)  7010094
431 .)  0348850
432 .)  3056250
433 .)  3216800
434 .)  5065500
435 .)  0276000
436 .)  5587000
437 .)  5507600
438 .)  5061000
439 .)  1610155
440 .)  1271320
441 .)  4213075
442 .)  1581810
443 .)  3329400
444 .)  3478400
445 .)  2102500
446 .)  3435500
447 .)  3364500
448 .)  7170990
449 .)  2106000
450 .)  3431000
451 .)  8057200
452 .)  3360000
453 .)  2331600
454 .)  4211550
455 .)  2038850
456 .)  8329000
457 .)  2231600
458 .)  1421770
459 .)  3353637
460 .)  2165000
461 .)  2487500
462 .)  3303000
463 .)  3456500
464 .)  2161500
465 .)  5592800
466 .)  6036805
467 .)  3307500
468 .)  2483000
469 .)  3072655
470 .)  6465700
471 .)  9128500
472 .)  3556500
473 .)  2061500
474 .)  1414250
475 .)  8130700
476 .)  3358685
477 .)  3207500
478 .)  1148900
479 .)  5063398
480 .)  8382650
481 .)  1077500
482 .)  3375800
483 .)  6665790
484 .)  6038800
485 .)  2385170
486 .)  4260500
487 .)  6033000
488 .)  1673800
489 .)  1111212
490 .)  2118000
491 .)  2429980
492 .)  